In [31]:
#from datasets import load_dataset
from torch.optim import Optimizer
import torch
import transformers
from torch.utils.tensorboard import SummaryWriter
from tqdm import tqdm
from transformers.models.llama.configuration_llama import LlamaConfig
from transformers.models.llama.modeling_llama import LlamaForCausalLM
from transformers import LlamaTokenizerFast
from functools import partial
from colossalai.nn.lr_scheduler import CosineAnnealingWarmupLR
from colossalai.nn.optimizer import HybridAdam

In [12]:
import pickle

In [13]:
with open('/data/rozen/home/e0833634/lama/protllama/uniprot_dataset.pkl', 'rb') as f:
    loaded_data = pickle.load(f)

In [14]:
loaded_data

DatasetDict({
    train: Dataset({
        features: ['attention_mask', 'input_ids', 'labels'],
        num_rows: 8716143
    })
    validation: Dataset({
        features: ['attention_mask', 'input_ids', 'labels'],
        num_rows: 1219069
    })
})

In [18]:
def tokenize_batch(batch, tokenizer: Optional[LlamaTokenizer] = None, max_length: int = 1024):
    texts = [sample['text'] for sample in batch]
    data = tokenizer(texts, return_tensors="pt", padding='max_length', truncation=True, max_length=max_length)
    data['labels'] = data['input_ids'].clone()
    return data

You are using the default legacy behaviour of the <class 'transformers.models.llama.tokenization_llama.LlamaTokenizer'>. If you see this, DO NOT PANIC! This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thouroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
normalizer.cc(51) LOG(INFO) precompiled_charsmap is empty. use identity normalization.


LlamaTokenizer(name_or_path='hf-internal-testing/llama-tokenizer', vocab_size=32000, model_max_length=2048, is_fast=False, padding_side='right', truncation_side='right', special_tokens={'bos_token': AddedToken("<s>", rstrip=False, lstrip=False, single_word=False, normalized=True), 'eos_token': AddedToken("</s>", rstrip=False, lstrip=False, single_word=False, normalized=True), 'unk_token': AddedToken("<unk>", rstrip=False, lstrip=False, single_word=False, normalized=True)}, clean_up_tokenization_spaces=False)

In [39]:
tokenizer_ = LlamaTokenizer.from_pretrained('hf-internal-testing/llama-tokenizer')
tokenizer_

AttributeError: 'tokenizers.Tokenizer' object has no attribute 'pad_token_id'

In [27]:
from tokenizers import Tokenizer
def load_tokenizer_with_special_tokens(path):
    # Load the tokenizer
    tokenizer = Tokenizer.from_file(path + 'tokenizer.json')

    # Set bos_token_id and eos_token_id to 0
    tokenizer.bos_token_id = 0
    tokenizer.eos_token_id = 0

    return tokenizer

path = '../../protgpt2/'  # replace with your directory
tokenizer = load_tokenizer_with_special_tokens(path)

# Confirming
assert tokenizer.bos_token_id == 0
assert tokenizer.eos_token_id == 0

In [40]:
tokenizer = LlamaTokenizerFast.from_pretrained('/data/rozen/home/e0833634/lama/protgpt2')

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'GPT2Tokenizer'. 
The class this function is called from is 'LlamaTokenizerFast'.


RecursionError: maximum recursion depth exceeded

In [33]:
MODEL_CONFIGS = {
    'protllama2': LlamaConfig(max_position_embeddings=1024, # maximum length
                      hidden_size = 1280,
                      bos_token_id = tokenizer.bos_token_id,
                      eos_token_id = tokenizer.eos_token_id,
                      transformers_version = transformers.__version__,
                      intermediate_size = 3440,
                      vocab_size = 50257),}
MODEL_CONFIGS

{'protllama2': LlamaConfig {
   "bos_token_id": 0,
   "eos_token_id": 0,
   "hidden_act": "silu",
   "hidden_size": 1280,
   "initializer_range": 0.02,
   "intermediate_size": 3440,
   "max_position_embeddings": 1024,
   "model_type": "llama",
   "num_attention_heads": 32,
   "num_hidden_layers": 32,
   "num_key_value_heads": 32,
   "pretraining_tp": 1,
   "rms_norm_eps": 1e-06,
   "rope_scaling": null,
   "rope_theta": 10000.0,
   "tie_word_embeddings": false,
   "transformers_version": "4.33.1",
   "use_cache": true,
   "vocab_size": 50257
 }}

In [ ]:
def save(booster: Booster, model: nn.Module, optimizer: Optimizer, lr_scheduler: _LRScheduler, epoch: int, step: int,
         batch_size: int, coordinator: DistCoordinator, save_dir: str):
    save_dir = os.path.join(save_dir, f'epoch{epoch}-step{step}')
    os.makedirs(os.path.join(save_dir, 'model'), exist_ok=True)

    booster.save_model(model, os.path.join(save_dir, 'model'), shard=True)
    booster.save_optimizer(optimizer, os.path.join(save_dir, 'optimizer'), shard=True)
    booster.save_lr_scheduler(lr_scheduler, os.path.join(save_dir, 'lr_scheduler'))
    running_states = {
        'epoch': epoch,
        'step': step,
        'sample_start_index': step * batch_size,
    }
    if coordinator.is_master():
        save_json(running_states, os.path.join(save_dir, 'running_states.json'))

In [1]:
from datasets import load_dataset
load_dataset('nferruz/UR50_2021_04')

KeyboardInterrupt: 

In [3]:
tokenizer = LlamaTokenizer.from_pretrained('nferruz/UR50_2021_04')

NameError: name 'LlamaTokenizer' is not defined

In [ ]:
load_dataset('togethercomputer/RedPajama-Data-1T-Sample')

In [ ]:
tokenizer = LlamaTokenizer.from_pretrained('hf-internal-testing/llama-tokenizer')
tokenizer.pad_token = tokenizer.unk_token

dataset = load_dataset('togethercomputer/RedPajama-Data-1T-Sample')
train_ds = dataset['train']
dataloader = prepare_dataloader(train_ds,
                                batch_size=2,
                                shuffle=True,
                                drop_last=True,
                                collate_fn=partial(tokenize_batch, tokenizer=tokenizer, max_length=4096))

In [ ]:
config = '7b'
config = MODEL_CONFIGS[config]

In [ ]:
model = LlamaForCausalLM(config)

In [ ]:
num_epochs = 1

In [ ]:
from colossalai.booster import Booster
from colossalai.booster.plugin import GeminiPlugin, LowLevelZeroPlugin

In [ ]:
optimizer = HybridAdam(model.parameters(), lr=3e-4, betas=(0.9, 0.95), weight_decay=0.1)
lr_scheduler = CosineAnnealingWarmupLR(optimizer,
                                    total_steps=num_epochs * len(dataloader),
                                    warmup_steps=2000,
                                    eta_min=0.1 * 3e-4)
#default_dtype = torch.float16 if args.mixed_precision == 'fp16' else torch.bfloat16
torch.set_default_dtype(torch.float16)
model, optimizer, _, dataloader, lr_scheduler = booster.boost(model,
                                                            optimizer,
                                                            dataloader=dataloader,
                                                            lr_scheduler=lr_scheduler)

In [ ]:
start_epoch = 0
start_step = 0
sampler_start_idx = 0
#if args.load is not None:
 #   coordinator.print_on_master('Loading checkpoint')
  #  start_epoch, start_step, sampler_start_idx = load(booster, model, optimizer, lr_scheduler, args.load)
   # coordinator.print_on_master(f'Loaded checkpoint {args.load} at epoch {start_epoch} step {start_step}')

num_steps_per_epoch = len(dataloader)
# if resume training, set the sampler start index to the correct value
dataloader.sampler.set_start_index(sampler_start_idx)
for epoch in range(start_epoch, args.num_epochs):
    dataloader.sampler.set_epoch(epoch)
    with tqdm(enumerate(dataloader),
                desc=f'Epoch {epoch}',
                disable=not coordinator.is_master(),
                total=num_steps_per_epoch,
                initial=start_step) as pbar:
        for step, batch in pbar:
            batch = {k: v.cuda() for k, v in batch.items()}
            outputs = model(**batch)
            loss = outputs[0]
            booster.backward(loss, optimizer)
            optimizer.step()
            lr_scheduler.step()
            optimizer.zero_grad()

            all_reduce_mean(loss)
            pbar.set_postfix({'loss': loss.item()})
            if coordinator.is_master():
                writer.add_scalar('loss', loss.item(), epoch * num_steps_per_epoch + step)

            if args.save_interval > 0 and (step + 1) % args.save_interval == 0:
                coordinator.print_on_master(f'Saving checkpoint')
                save(booster, model, optimizer, lr_scheduler, epoch, step + 1, args.batch_size, coordinator,
                        args.save_dir)
                coordinator.print_on_master(f'Saved checkpoint at epoch {epoch} step {step + 1}')
    # the continue epochs are not resumed, so we need to reset the sampler start index and start step
    dataloader.sampler.set_start_index(0)
    start_step = 0

coordinator.print_on_master(f'Max CUDA memory usage: {torch.cuda.max_memory_allocated()/1024**2:.2f} MB')